# EDA

path_plot = '/home/petra42/GIT/aida_question_classification/plots/'

## Libaries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

#visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#nltk   import für eda_1 - preprozessing dataframe
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# default setting for lemmatizer and stopwords
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# remove stop words
nltk.download('stopwords')
my_stopwords = set(stopwords.words('english'))

##libaries for part od speech

In [ ]:
import spacy
import csv
nlp = spacy.load("en_core_web_sm")
#plot spacy
from spacy import displacy
from spacy.attrs import ORTH

In [ ]:
from nltk import pos_tag
from collections import Counter
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

from nltk import pos_tag
from nltk import FreqDist
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk import FreqDist

##Functions

In [ ]:
# from folder data
# import data.get_data.process_question(row)
def process_question(row):
   '''join row to text''' 
   text =  " ".join(row.split(" ")[1:])
   return text

In [ ]:
def get_plot_length_text(df, title, diagram):
    '''
    creates a histogram 
    that shows the distribution of the number of tokens 
    per line for the data set to be examined
    
    Parameters
    ----------
    df:
        series; to examined text
    title: 
        string; title for Plot
    diagram: 
        string: name of the diagram
    
    Returns
    -------
    Vizualisationplot Distribution
    '''
    plt.hist(df.apply(lambda text: len(text.split())))
    plt.xlabel('number of token')
    plt.ylabel('frequency')
    plt.title(title)
    plt.savefig(path_plot + diagram + '.png')
    plt.show()
    
    avg = round(df.apply(lambda text: len(text.split())).mean())
    maxi = max(list(df.apply(lambda text: len(text.split()))))
    mini = min(list(df.apply(lambda text: len(text.split()))))
    
    print(f'The texts have a mean length of {avg} tokens.')
    print(f'The longest text has {maxi} tokens, the shortest has {mini} tokens.')

In [ ]:
def corpus_func(df):
    '''
    create a textcorpus from pd.series, df['column']

    Parameters
    ----------
    text, string

    Returns
    ------_
    concatinated string with marker --XXX-- as selector
    '''
    return " XXX ".join(text for text in df)

def split_corpus_func(corpus):
    '''
    create a column from text corpus with marker ' XXX '
    as selector

    Parameters
    ----------
    text, string

    Returns
    ------_
    list of text 
    '''
    list = lambda x: x.split(' XXX ')
    return list

def preprocess_dataframe(df):
    '''
    create new columns in the data frame
    new_column: 'text' => cleaned stopwords (english)
                'text_clean' => regex, lowercase
                'text_lemma' => lemmetized
    param: df
    returns: df with new columns
    '''

    corpus = corpus_func(df['question'])
    text_corpus = stopword_text(corpus)
    df['text'] = split_corpus_func(corpus)
    clean_corpus = clean_text(text_corpus)
    df['text_clean'] = split_corpus_func(clean_corpus)
    lemma = lem_text(clean_corpus)
    df['text_lemma'] = split_corpus_func(clean_corpus)
    return df


In [ ]:
#import utils.text_manipulation

def clean_text(text):
    """
    Regex cleaning of the text. Filters everything except alphanumerical and '.
    Return is turned into lower case

    Parameters
    ----------
    text : string
        text to be cleaned

    Returns
    -------
    string
        lower case regex cleaned text

    """
    text = text.replace("´", "'")
    
    text = text.replace("'s", " ")

    digi_punct = "[^a-zA-Z.1234567890#' ]"
    text = re.sub(digi_punct, " ", text)
    text = " ".join(text.split())
    text = text.lower()
    return text

def stopword_text(text):
    """
    Remove all words in the text that are in the stopword list
     
    Parameters
    ----------
    text : string
    
    Returns
    -------
    string
        text only stopwords
     
    """

    return " ".join([word for word in text.split() if word not in my_stopwords])

def lem_text(text):
    """
    Group the different inflected forms of a word so they can be analysed as 
    a single item
     
    Parameters
    ----------
    text : string
    
    Returns
    -------
    string
        text with lemmas
    """

    lem_sentence = text.split()
    for i, word in enumerate(text.split()):
        for pos in "n", "v", "a", "r":
            lem = lemmatizer.lemmatize(word, pos=pos)
            if lem != word:
                lem_sentence[i] = lem
                break
            else:
                lem_sentence[i] = word
    return " ".join(lem_sentence)


# Load data

In [ ]:
# data
path_class_def  = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/definition.html'
path_train_data = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label'
path_test_data  = 'https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label'

In [ ]:
#load
train_df = pd.read_table(path_train_data, encoding = "ISO-8859-1", header=None)
train_df.columns = ["raw"]
train_df['category'] = train_df.apply (lambda row: row["raw"].split(":")[0], axis=1)
train_df['subcategory'] = train_df.apply (lambda row: row["raw"].split(" ")[0].split(":")[1], axis=1)
train_df['question'] = train_df.apply (lambda row: process_question(row["raw"]), axis=1)

train_df.head(2)

In [ ]:
test_df = pd.read_table(path_test_data, encoding = "ISO-8859-1", header=None)
test_df.columns = ["raw"]
test_df['category'] = train_df.apply (lambda row: row["raw"].split(":")[0], axis=1)
test_df['subcategory'] = train_df.apply (lambda row: row["raw"].split(" ")[0].split(":")[1], axis=1)
test_df['question'] = train_df.apply (lambda row: process_question(row["raw"]), axis=1)

test_df.head(2)

#corpus

In [ ]:
# corpus from df_train['question'] 
corpus = corpus_func(train_df['question'])
print(corpus[:500])

In [ ]:
#corpus from df_test['question']
corpus_test = corpus_func(test_df['question'])
print(corpus_test[:500])

In [ ]:
text = corpus

#sample
text_sample = process_question(train_df['raw'][10])


#Part of speech tags

In [ ]:
# Split the data into individual tokens.
text_splitted = text.split()


# or:
word_tokens = word_tokenize(text) 

print('split:\t',text_splitted[:100],'\ntokenize:',word_tokens[:100])

In [ ]:
# Tag each token into a specific part of speech 
word_tagged = nltk.pos_tag(word_tokens)

print('tagged:\t',word_tagged[:10])

In [ ]:
# Print out the frequency for each part of speech. (i.e. 2 NN's, 100 VBG's, etc.)
# To see the acronym definitions for each part of speech,
# look at the following link: https://pythonprogramming.net/part-of-speech-tagging-nltk-tutorial/

# with counter from collections
word_count = sorted(Counter(word_tagged).items(), key = lambda kv:( kv[1], kv[0]), reverse=True)

#or with nltk FreqDist
fdist = FreqDist(word_tagged)
first20_count = fdist.most_common(10)


print(f'word_count:{word_count}\n len word_count: {len(word_count)}')
print(f'first20_count :{first20_count }')

In [ ]:
doc = nlp(text_sample)

print('token','-', 'lemma_', '-', 'lower_''-', 'pos_','-', 'tag_-', 'dep_','-', 'sentiment','-','is_alpha','-', 'is_stop')

for token in doc:
    #any token_attributs; more: https://spacy.io/api/token#attributes
    
    print('------------------------------------------------------------------------\n')   
    print(token.text,'-', token.lemma_, '-', token.lower_,'-', token.pos_,'-', token.tag_,'-',
          token.dep_,'-', token.sentiment,'-', token.is_alpha,'-', token.is_stop)

'''
Text: The original word text.
Lemma: The base form of the word.
POS: The simple UPOS part-of-speech tag.
Tag: The detailed part-of-speech tag.
Dep: Syntactic dependency, i.e. the relation between tokens.
Shape: The word shape – capitalization, punctuation, digits.
is alpha: Is the token an alpha character?
is stop: Is the token part of a stop list, i.e. the most common words of the language?'''

In [ ]:
# Print out the entire spacy text.   --- split by 300 ...to long for display
print(doc[:300])

In [ ]:
# check doc is tagged?
print(doc.is_tagged)

In [ ]:
#token_tags + explain
print([(token.text, token.tag_, spacy.explain(token.tag_)) for token in doc])

In [ ]:
#_entities + label
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
#token_dep_ + explain
print([(token.text, token.dep_, spacy.explain(token.dep_)) for token in doc])

spaCy add on:
doc, any attributes, chunk

In [ ]:
#any token_attributs; more: https://spacy.io/api/token#attributes
print('token','-', 'lemma_', '-', 'lower_''-', 'pos_','-', 'tag_-', 'dep_','-', 'sentiment','-','is_alpha','-', 'is_stop')
print('------------------------------------------------------------------------')   
for i, token in zip(range (10), doc):
    print(i, token.text,'-', token.lemma_, '-', token.lower_,'-', token.pos_,'-', token.tag_,'-',
          token.dep_,'-', token.sentiment,'-', token.is_alpha,'-', token.is_stop)

In [ ]:
#Base Noun Phrases - flat phrases whose head is based on a noun
for chunk in doc[0:10].noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

In [ ]:
# For the first 100 words, print out the text, tag, and explanation of the tag.

# If you want to format things better, try using f strings! For example, look at the following:
# for number in range(0, 100):
#   print(f"Original Number: {number}, {' ':{10}} Original Number times 2: {number * 10}") 

i = 0
for i, token in zip(range(99),doc):
    print(f"{token}\t{' ':{7}}\t{token.tag_}\t\t{' ':{8}}{spacy.explain(token.tag_)}")

# {' ':{10} or \t ---> it won't be very nice

In [ ]:
#Count the frequencies of a given attribute with ORTH
print(doc.count_by(ORTH))
print(len(doc.count_by(ORTH)))

In [ ]:
#other way

dict_tags={}
for token in doc:
  #print(token,token.tag_)
  try:
    dict_tags[token.tag_].append(token.text)
  except:
    dict_tags[token.tag_]=[token.text]

print(dict_tags)

In [ ]:
tag_labels = []
for key in doc:
    while token.head != token:
        tag_labels.append(token.tag_)
        token = token.head
print(tag_labels)

In [ ]:
tags_count=[]
for value in dict_tags.values():
    tags_count.append(len(value))
print(tags_count) 

In [ ]:
# Visualize the POS tags of the first sentence in the original text file with displacy.

#sentence
sents = list(doc.sents)
print(len(sents))
sents[0]
print(sents[0])

displacy.render(sents[0],style="dep" ,jupyter=True, options = {'distance' : 100})
plt.savefig(path_plot + 'POS_tag.png')


In [ ]:
# Visualize the Key_words and your classification of the first sentence with displacy.
displacy.render(sents[0],style="ent" ,jupyter=True)
plt.savefig(path_plot + diagram + '.png')

copy from:
https://shirishkadam.com/2017/07/03/nlp-question-classification-using-support-vector-machines-spacyscikit-learnpandas/

In [ ]:
def process_question (question, qclass, en_nlp):
    en_doc = nlp (u, ' ' + question)
    sent_list = list (en_doc.sents) 
    sent = sent_list [0] 
    wh_bi_gram = [] 
    root_token = "" 
    wh_pos = "" 
    wh_nbor_pos = "" 
    wh_word = "" 
    for sent in token:
        if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP $" or token.tag_ == "WRB":
            wh_pos = token .tag_ 
            wh_word = token.text 
            wh_bi_gram.append (token.text) 
            wh_bi_gram.append (str (en_doc [token.i + 1])) 
            wh_nbor_pos = en_doc [token.i + 1].tag_ 
        if token.dep_ == "ROOT":
            root_token = token.tag_ 

    write_each_record_to_csv (wh_pos, wh_word, wh_bi_gram, wh_nbor_pos, root_token)

process_question (df_train['question'][50])